<a href="https://colab.research.google.com/github/Aryabhatt-O/Amartya-Roy/blob/main/Voronoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Voronoi Implemenation**
[qhull](http://www.qhull.org/html/index.htm)

[scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.Voronoi.html)

[*Relationship between Voronoi  and Delanauy triangulations*](https://medium.com/stamatics-iit-kanpur/voronoi-diagrams-and-delaunay-triangulations-cc57ba901f9e)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
points = np.array([[0,0],[2,2],[-2,2]])
from scipy.spatial import Voronoi, voronoi_plot_2d
vor = Voronoi(points)
fig = voronoi_plot_2d(vor)
plt.show()
vor.ridge_points


In [ ]:
pip install openmesh

In [ ]:
from PIL import Image
import random
import math
 
def generate_voronoi_diagram(width, height, num_cells):
  image = Image.new("RGB", (width, height))
  putpixel = image.putpixel
  imgx, imgy = image.size
  nx = []
  ny = []
  nr = []
  ng = []
  nb = []
  for i in range(num_cells):
    nx.append(random.randrange(imgx))
    ny.append(random.randrange(imgy))
    nr.append(random.randrange(256))
    ng.append(random.randrange(256))
    nb.append(random.randrange(256))
  for y in range(imgy):
    for x in range(imgx):
      dmin = math.hypot(imgx-1, imgy-1)
      j = -1
      for i in range(num_cells):
        d = math.hypot(nx[i]-x, ny[i]-y)
        if d < dmin:
          dmin = d
          j = i
          putpixel((x, y), (nr[j], ng[j], nb[j]))
          image.save("VoronoiDiagram.png", "PNG")
          image.show()
 
generate_voronoi_diagram(500, 500, 25)

In [ ]:
import math as m

# Utils
def findHAngle(dx, dy):
  """Determines the angle with respect to the x axis of a segment
  of coordinates dx and dy
  """
  l = m.sqrt(dx*dx + dy*dy)
  if dy > 0:
    return m.acos(dx/l)
  else:
    return 2*m.pi - m.acos(dx/l)


class Vertex:
  def __init__(self, x, y):
    self.x = x
    self.y = y
    self.hedges = []  # list of halfedges whose tail is this vertex

  def __eq__(self, other):
    if isinstance(other, Vertex):
      return self.x == other.x and self.y == other.y
    return NotImplemented

  def sortHedges(self):
    self.hedges.sort(key=lambda a: a.angle, reverse=True)

  def __repr__(self):
    return "({0},{1})".format(self.x, self.y)


class Hedge:
  # v1 -> v2
  def __init__(self, v1, v2):
    self.prev = None
    self.twin = None
    self.next = None
    self.tail = v1
    self.face = None
    self.angle = findHAngle(v2.x-v1.x, v2.y-v1.y)

  def __eq__(self, other):
    return self.tail == other.tail and \
        self.next.tail == other.next.tail

  def __repr__(self):
    if self.next is not None:
      return "({0},{1})->({2},{3})".format(self.tail.x, self.tail.y,
                                           self.next.tail.x,
                                           self.next.tail.y)
    else:
      return "({0},{1})->()".format(self.tail.x, self.tail.y)


class Face:
  def __init__(self):
    self.halfEdge = None
    self.name = None


class DCEL:
  def __init__(self):
    self.vertices = []
    self.hedges = []
    self.faces = []

  # Returns vertex object given x and y
  def findVertex(self, x, y):
    for v in self.vertices:
      if v.x == x and v.y == y:
        return v
    return None

  # Returns Halfedge whole vertices are v1 and v2
  # v1 and v2 are tuples
  def findHalfEdge(self, v1, v2):
    for halfEdge in self.hedges:
      nextEdge = halfEdge.next
      if (halfEdge.tail.x == v1[0] and halfEdge.tail.y == v1[1]) and (nextEdge.tail.x == v2[0] and nextEdge.tail.y == v2[1]):
        return halfEdge

    return None

  def build_dcel(self, points, segments):

    #  For each point create a vertex and add it to vertices
    for point in points:
      self.vertices.append(Vertex(point[0], point[1]))

    # For each input segment, create to hedges and assign their
    # tail vertices and twins

    # Structures of segment is [(0, 5), (2, 5)]
    for segment in segments:
      startVertex = segment[0]
      endVertex = segment[1]

      v1 = self.findVertex(startVertex[0], startVertex[1])
      v2 = self.findVertex(endVertex[0], endVertex[1])

      h1 = Hedge(v1, v2)
      h2 = Hedge(v2, v1)

      h1.twin = h2
      h2.twin = h1

      v1.hedges.append(h1)
      v2.hedges.append(h2)

      self.hedges.append(h1)
      self.hedges.append(h2)

    # For each endpoint, sort the half-edges whose
    # tail vertex is that endpoint in clockwise order.

    for vertex in self.vertices:
      vertex.sortHedges()

      noOfHalfEdges = len(vertex.hedges)

      if noOfHalfEdges < 2:
        return Exception("Invalid DCEL. There should be at least two half edges for a vertex")

      # For every pair of half-edges e1, e2 in clockwise order,
      # assign e1->twin->next = e2 and e2->prev = e1->twin.
      for i in range(noOfHalfEdges - 1):
        e1 = vertex.hedges[i]
        e2 = vertex.hedges[i+1]

        e1.twin.next = e2
        e2.prev = e1.twin

      # for the last and first halfedges pair
      e1 = vertex.hedges[noOfHalfEdges - 1]
      e2 = vertex.hedges[0]

      e1.twin.next = e2
      e2.prev = e1.twin

    # For every cycle, allocate and assign a face structure.
    faceCount = 0
    for halfEdge in self.hedges:

      if halfEdge.face == None:
        print('here')
        faceCount += 1

        f = Face()
        f.name = "f" + str(faceCount)

        f.halfEdge = halfEdge
        halfEdge.face = f

        h = halfEdge
        while (not h.next == halfEdge):
          h.face = f
          h = h.next
        h.face = f

        self.faces.append(f)

  # Given a half, find all the regions
  # The format of segment is [(0, 5), (2, 5)]

  def findRegionGivenSegment(self, segment):
    # We need to find the half edge whose vertices
    # are that of the passed segment
    v1 = segment[0]
    v2 = segment[1]
    startEdge = self.findHalfEdge(v1, v2)

    h = startEdge
    while (not h.next == startEdge):
      print(h, end="--->")
      h = h.next
    print(h, '--->', startEdge)


def main():
  points = [(0, 5), (2, 5), (3, 0), (0, 0)]

  segments = [
      [(0, 5), (2, 5)],
      [(2, 5), (3, 0)],
      [(3, 0), (0, 0)],
      [(0, 0), (0, 5)],
      [(0, 5), (3, 0)],
  ]

  myDCEL = DCEL()
  myDCEL.build_dcel(points, segments)

  myDCEL.findRegionGivenSegment([(3, 0), (0, 5)])


main()


# class point(self,x,y,type):
  

# class half_edge(p1,p2,sym):
  # half


In [ ]:
;
  
from distutils.core import setup
setup(name = 'dcel',
    description = "implementation of a doubly connected edge list",
    version = '0.1.1',
    author = "Angel Yanguas-Gil",
    author_email = "angel.yanguas@gmail.com",
    url = "http://ayanguasgil.net/",
    download_url = (
        "http://ayanguasgil.net/stuff/dcel-0.1.1.tar.gz"),
    packages = ['dcel'],
    classifiers = ["Development Status :: 2 - Pre-Alpha",
        "Environment :: Console",
        "Topic :: Scientific/Engineering",
        "Topic :: Utilities",
        "License :: OSI Approved :: BSD License" ])


In [ ]:
from random import randint
from delaunator import Delaunator, TrianguledObject

In [ ]:
# My  approach:
# 1.Find all ridges around the point I want, using vor.ridge_points
# 2.Take all of the ridge vertices from these ridges, as a set
# 3.Look for the (unique) region with the same set of vertices.


def restro(points):
  M = 30
  # points = np.random.uniform(0, 100, size=(M, 2))
  vor = Voronoi(points)
  voronoi_plot_2d(vor)


def myFriends(new_point):
  # new_point = [50, 50]
  plt.plot(new_point[1], new_point[1], 'ro')

point_index = np.argmin(np.sum((points - new_point)**2, axis=1))
ridges = np.where(vor.ridge_points == point_index)[0]
vertex_set = set(np.array(vor.ridge_vertices)[ridges, :].ravel())
region = [x for x in vor.regions if set(x) == vertex_set][0]

polygon = vor.vertices[region]
plt.fill(*zip(*polygon), color='yellow')  
plt.show()

In [ ]:
pip install pydelaunator

In [ ]:
import numpy as np
y,x = np.meshgrid(range(10),range(20))



In [ ]:
pip install MeshPy

In [ ]:
from meshpy.tet import MeshInfo, build

In [ ]:
import numpy as np
S = 15
points = np.random.uniform(0, 100, size=(S, 2))
vor = Voronoi(points)
voronoi_plot_2d(vor)

new_point = [20, 30]   
plt.plot(new_point[0], new_point[1], 'ro')

point_index = np.argmin(np.sum((points - new_point)**2, axis=1))
print(point_index)
ridges = np.where(vor.ridge_points == point_index)[0] 
print(ridges)   #Indices of the points between which each Voronoi ridge lies.
vertex_set = set(np.array(vor.ridge_vertices)[ridges, :].ravel())
print(vertex_set) #Indices of the Voronoi vertices forming each Voronoi ridge.
region = [x for x in vor.regions if set(x) == vertex_set][0]
print(region)

polygon = vor.vertices[region]
print(polygon)
plt.fill(*zip(*polygon), color='yellow')  
plt.show()

In [ ]:
!pip install CGAL
import CGAL.Kernel.Point_2;

#  **Point is in which region of voronoi diagram**

In [ ]:
def restro(points):
  vor = Voronoi(points)
  voronoi_plot_2d(vor)

def friends(new_point):
  plt.plot(new_point[0], new_point[1], 'ro')
                                                         
def find_Restro():
  S = 15
  points = np.random.uniform(0, 100, size=(S, 2))
  new_point = [20,30]  
  # point_index = np.argmin(np.sum((points - new_point)**2, axis=1))
  point_index = vor.point_region
  print(point_index)
  ridges = np.where(vor.ridge_points == point_index)[0]
  vertex_set = set(np.array(vor.ridge_vertices)[ridges, :].ravel())
  region = [x for x in vor.regions if set(x) == vertex_set][0]
  polygon = vor.vertices[region]
  


def hey_locate(restro,friends,find_Restro):
  return(restro(points),friends(new_point),find_Restro())



if __name__ == "__main__":
  hey_locate(restro,friends,find_Restro)
  plt.fill(*zip(*polygon), color='yellow')
  plt.show()





In [ ]:
def commentchar():
    """returns a list with the chars that signal comment lines"""

    return _commentchars

def addcommentchar(c):
    """appends a char to the list of chars that signal comment lines"""

    if not c in _commentchars:
        _commentchars.append(c[0])

def setcommentchar(lc):
    """passes a list of comment markers"""

    _commentchars = lc

def tokenize(line):
    """determines the tokens inside a line (space or tab
    separated) and returns their numerical values or, if that
    fails, the token as a string"""

    lw = []
    if len(line) == 0 or line[0] in _commentchars:
        return lw
    else:
        for word in line.split():
            try:
                val = float(word)
            except:
                val = word

            lw.append(val)
        return lw


def filetosheet(filename):
    """Given a file, returns a sheet. A sheet is basically a list
    of data rows"""

    file = open(filename, 'r')
    lines = file.readlines()
    file.close()
    plist = []
    for line in lines:
        p = tokenize(line)
        if len(p) == 0:
            continue
        plist.append(p)
    return plist

def extractcolumn(sheet, coln):
    """gets column from sheet"""

    col = []
    for row in sheet:
        try:
            v = row[coln-1]
        except:
            v = _emptycell
        col.append(v)
    return col

def operate(col1, col2, func):
    """Just another name for map(f,l1,l2)"""

    return map(func, col1, col2)

def transform(col, func):
    """Just another name for map(f,l)"""

    return map(func, col)

In [ ]:
import math as m

import iodata as tok
# import pyeps as ps


class Xygraph:
    """Represents a set of vertices connected by undirected edges.
    The vertices are stored in a list of coordinates, while
    the edges are stored as a pair of indices (i,j) of the vertices
    list.
    """

    def __init__(self, vl=[], el=[]):
        """Creates the 2D graph formed by a list of vertices (x,y)
        and a list of indices (i,j)
        """
        self.vl = vl
        self.el = el
        if self.vl != []:
            self.minmax()

    def minmax(self):
        """Determines the boundary box of the vertices in the graph"""
        vx = [v[0] for v in self.vl]
        vy = [v[1] for v in self.vl]
        self.xmax, self.xmin = max(vx), min(vx)
        self.ymax, self.ymin = max(vy), min(vy)

    def clip(self, clipbox):
        """Trims the vertex and edge list of elements that lie
        outside a clipping box [(xmin,xmax),(ymin,ymax)]"""
        pmin, pmax = clipbox
        ind = []
        vlt = []
        #Direct elimination of out of bounds edges and vertices
        for i in range(len(self.vl)):
            if self.vl[i][0] < pmin[0] or self.vl[i][1] < pmin[1] or \
                self.vl[i][0] > pmax[0] or self.vl[i][1] > pmax[1]:
                ind.append(i)
            else:
                vlt.append(self.vl[i])
        elt = filter((lambda x:(x[0] not in ind) and (x[1] not in ind)),
            self.el)
        li = filter((lambda x: x not in ind),range(len(self.vl)))
        #We rename the indices in the trimmed edge list
        lf = range(len(self.vl) - len(ind))
        equiv = {}
        for i in range(len(li)):
            if li[i] != lf[i]:
                equiv[li[i]] = lf[i]

        for i in range(len(elt)):
            if elt[i][0] in equiv:
                x = equiv[elt[i][0]]
            else:
                x = elt[i][0]
            if elt[i][1] in equiv:
                y = equiv[elt[i][1]]
            else:
                y = elt[i][1]
            elt[i] = (x,y)

        self.vl = vlt
        self.el = elt
        self.minmax()

    def load(self, filename):
        """loads a xygraph from filename. The structure of the
        file should be that given by save method.
        """

        data = tok.filetosheet(filename)
        if data is not None:
            nv = data[0][0]
            self.vl = data[1:nv+1]
            self.el = data[nv+1:]
            self.minmax()

    def save(self, filename):
        """saves a xygraph to filename"""
        file = open(filename,'w')
        file.write("%d\n" % len(self.vl))
        for v in self.vl:
            file.write("%f %f\n" % (v[0], v[1]))
        for e in self.el:
            file.write("%d %d\n" % e)
        file.close()

    def saveplot(self, filename=None, res=512):
        """
        Creates a PS representation of the xygraph. Saves
        the plot as an EPS file is filename is provided.
        """
        canvas = []
        offset = 50
        dx = self.xmax - self.xmin
        dy = self.ymax - self.ymin
        dl = max(dx, dy)
        r = float(res) / dl
        for i, j in self.el:
            x0 = int(r*(self.vl[i][0] - self.xmin)) + offset
            y0 = int(r*(self.vl[i][1] - self.ymin)) + offset
            x1 = int(r*(self.vl[j][0] - self.xmin)) + offset
            y1 = int(r*(self.vl[j][1] - self.ymin)) + offset
            canvas.append(ps.PSLine((x0,y0), (x1,y1)))
        up = res + 2*offset
        right = res + 2*offset
        fb = offset
        fu = res + offset
        frame = ps.PSPolygon([(fb, fb), (fu, fb), \
            (fu, fu), (fb, fu)], 1)
        canvas.append(frame)

        plot = ps.PSPlot(canvas)
        bound = ps.PSClip([(0, 0), (0, right),
            (up, 0), (up, right)])
        plot.setbound(bound)
        if filename is not None:
            plot.saveeps(filename)
        return plot

if __name__=='__main__':
    import sys
    g = Xygraph()
    if len(sys.argv) < 2:
        print("Use: xygraph filename")
    else:
        g.load(sys.argv[1])
        g.saveplot("newplot.eps")

In [ ]:
import numpy
from PIL import Image

def voronoi(points,shape=(500,500)):
    depthmap = numpy.ones(shape,numpy.float)*1e308
    colormap = numpy.zeros(shape,numpy.int)

    def hypot(X,Y):
        return (X-x)**2 + (Y-y)**2

    for i,(x,y) in enumerate(points):
        paraboloid = numpy.fromfunction(hypot,shape)
        colormap = numpy.where(paraboloid < depthmap,i+1,colormap)
        depthmap = numpy.where(paraboloid <
depthmap,paraboloid,depthmap)

    for (x,y) in points:
        colormap[x-1:x+2,y-1:y+2] = 0

    return colormap

def draw_map(colormap):
    shape = colormap.shape

    palette = numpy.array([
            0x000000FF,
            0xFF0000FF,
            0x00FF00FF,
            0xFFFF00FF,
            0x0000FFFF,
            0xFF00FFFF,
            0x00FFFFFF,
            0xFFFFFFFF,
            ])

    colormap = numpy.transpose(colormap)
    pixels = numpy.empty(colormap.shape+(4,),numpy.int8)

    pixels[:,:,3] = palette[colormap] & 0xFF
    pixels[:,:,2] = (palette[colormap]>>8) & 0xFF
    pixels[:,:,1] = (palette[colormap]>>16) & 0xFF
    pixels[:,:,0] = (palette[colormap]>>24) & 0xFF

    image = Image.frombytes("RGBA",shape,pixels)
    image.save('voronoi.png')

if __name__ == '__main__':
    draw_map(voronoi(([100,100],[356,301],[400,65],[324,145],[200,399])))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# points = np.random.rand(10,2) #random
points = np.array([[5, 10], [0, 1], [0, 2], [1, 0], [-6, 7], [-3, 2],[2, 0], [12, 1], [2, 20]])
from scipy.spatial import Voronoi, voronoi_plot_2d
vor = Voronoi(points)
fig = voronoi_plot_2d(vor)
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',line_width=3, line_alpha=0.6, point_size=2)
plt.show()

from scipy.spatial import KDTree
# points = np.array([[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [3,2 ],[2, 0], [2, 1], [2, 2]])
tree = KDTree(points)
tree.query([4,2]) 

In [ ]:
pip install openmesh

In [ ]:
!pip install gevent --pre
!pip install auto-py-to-exe
!pip install pydelaunator

In [ ]:
pip install --upgrade setuptools pip

In [ ]:
pip install sklearn-geometry

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d


M = 10

Matrix = [(random.random()*100,random.random()*100)  for x in range(M)]
points = np.array(Matrix)


vor = Voronoi(points)
# print(vor.ridge_vertices)

voronoi_plot_2d(vor)
plt.show()
print(vor.points)
vor.point_region

In [ ]:
!pip install pydelaunator

In [ ]:
!pip install pyglet --user

In [ ]:
!pip install pydelaunator

In [ ]:
!git clone https://github.com/khuyentran1401/Voronoi-diagram.git!
!cd Voronoi-diagram

In [ ]:

from __future__ import print_function
from CGAL.CGAL_Kernel import Point_2
from CGAL.CGAL_Voronoi_diagram_2 import Voronoi_diagram_2
from CGAL.CGAL_Voronoi_diagram_2 import Voronoi_diagram_2_Halfedge_handle
from CGAL.CGAL_Voronoi_diagram_2 import Voronoi_diagram_2_Vertex_handle
from CGAL.CGAL_Voronoi_diagram_2 import Voronoi_diagram_2_Face_handle
from CGAL.CGAL_Voronoi_diagram_2 import Voronoi_diagram_2_Ccb_halfedge_circulator
from CGAL.CGAL_Voronoi_diagram_2 import Voronoi_diagram_2_Locate_result


def print_endpoint(e, is_src):
    print("\t", end='')
    if is_src:
        if e.has_source():
            print(e.source().point())
        else:
            print("point at infinity")
    else:
        if e.has_target():
            print(e.target().point())
        else:
            print("point at infinity")


points = []
points.append(Point_2(0, 0))
points.append(Point_2(100, 0))
points.append(Point_2(100, 100))
points.append(Point_2(0, 100))
points.append(Point_2(200, 0))
points.append(Point_2(300, 0))
points.append(Point_2(350, 0))

vd = Voronoi_diagram_2(points)

assert(vd.is_valid())


queries = []
queries.append(Point_2(0, 0))
queries.append(Point_2(50, 50))
queries.append(Point_2(0, 50))
queries.append(Point_2(50, 40))
queries.append(Point_2(10, 20))
queries.append(Point_2(150, 0))
queries.append(Point_2(150, 200))
queries.append(Point_2(200, 0))
queries.append(Point_2(250, 0))
queries.append(Point_2(300, 0))
queries.append(Point_2(325, 10))

for p in queries:
    print("Query point (", p, ") lies on a Voronoi", end='')
    lr = vd.locate(p)
    if lr.is_vertex_handle():
        v = lr.get_vertex_handle()
        print("vertex.")
        print("The Voronoi vertex is:\t")
        print(v.point())
    else:
        if lr.is_halfedge_handle():
            e = lr.get_halfedge_handle()
            print("edge.")
            print("The source and target vertices of the Voronoi edge are:")
            print_endpoint(e, True)
            print_endpoint(e, False)
        else:
            if lr.is_face_handle():
                f = lr.get_face_handle()
                print("face.")
                print("The vertices of the Voronoi face are (in counterclockwise order):")
                ec_start = f.outer_ccb()

                if ec_start.hasNext():
                    done = ec_start.next()
                    iter = Voronoi_diagram_2_Halfedge_handle()
                    while(1):
                        iter = ec_start.next()
                        print_endpoint(iter, False)
                        if iter == done:
                            break
        print("")

In [ ]:
pip install pydelaunator